# Day 6: Guard Gallivant

[https://adventofcode.com/2024/day/6](https://adventofcode.com/2024/day/6)

## Description

### Part One

The Historians use their fancy [device](https://adventofcode.com/2024/day/4) again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year [1518](https://adventofcode.com/2018/day/5)! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single _guard_ is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

    ....#.....
    .........#
    ..........
    ..#.......
    .......#..
    ..........
    .#..^.....
    ........#.
    #.........
    ......#...

The map shows the current position of the guard with `^` (to indicate the guard is currently facing _up_ from the perspective of the map). Any _obstructions_ - crates, desks, alchemical reactors, etc. - are shown as `#`.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

* If there is something directly in front of you, turn right 90 degrees.
* Otherwise, take a step forward.

Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

    ....#.....
    ....^....#
    ..........
    ..#.......
    .......#..
    ..........
    .#........
    ........#.
    #.........
    ......#...

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

    ....#.....
    ........>#
    ..........
    ..#.......
    .......#..
    ..........
    .#........
    ........#.
    #.........
    ......#...

Reaching another obstacle (a spool of several _very_ long polymers), she turns right again and continues downward:

    ....#.....
    .........#
    ..........
    ..#.......
    .......#..
    ..........
    .#......v.
    ........#.
    #.........
    ......#...

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

    ....#.....
    .........#
    ..........
    ..#.......
    .......#..
    ..........
    .#........
    ........#.
    #.........
    ......#v..

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. _Including the guard's starting position_, the positions visited by the guard before leaving the area are marked with an `X`:

    ....#.....
    ....XXXXX#
    ....X...X.
    ..#.X...X.
    ..XXXXX#X.
    ..X.X.X.X.
    .#XXXXXXX.
    .XXXXXXX#.
    #XXXXXXX..
    ......#X..

In this example, the guard will visit _`41`_ distinct positions on your map.

Predict the path of the guard. _How many distinct positions will the guard visit before leaving the mapped area?_

#### Example

In [827]:
def read_input(filename):
    """Reads a grid file and identifies the starting position and direction.

    This function reads a file containing a grid of characters and identifies a starting position
    marked by '^'. The starting position is then converted to an empty space ('.') in the grid.

    Args:
        filename (str): Path to the input file containing the grid.

    Returns:
        tuple: A tuple containing three elements:
            - list[list[str]]: The grid represented as a 2D list of characters
            - tuple[int, int] or None: The starting position as (row, col), or None if not found
            - str or None: The starting direction ('^'), or None if not found

    Example:
        >>> grid, start_pos, start_dir = read_input('input.txt')
        >>> print(start_pos)
        (0, 1)
        >>> print(start_dir)
        '^'
    """
    grid = []
    start_pos = None
    start_dir = None
    start_obj = None  # todo what is in the floor, if its an obstacle etc

    with open(filename) as f:
        for row, line in enumerate(f):
            row_data = list(line.strip())
            # Find starting position if it exists in this row
            if "^" in row_data:
                col = row_data.index("^")
                start_pos = (row, col)
                start_dir = "^"
                row_data[col] = "."  # Replace start position with empty space
                start_obj = row_data[col]
            grid.append(row_data)

    return grid, start_pos, start_dir, start_obj


# Read the input file
grid, start_pos, start_dir, start_obj = read_input("example.txt")

In [828]:
grid[start_pos[0]][start_pos[1]]

'.'

In [829]:
# turning 90 degrees just by going up the index
directions = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}

In [830]:
def change_direction(direction):
    """
    Changes the current direction to the next direction in a clockwise rotation.

    The rotation follows the pattern: up (^) -> right (>) -> down (v) -> left (<) -> up (^)

    Args:
        direction (str): The current direction symbol (^, >, v, or <)

    Returns:
        str: The next direction symbol in the clockwise rotation

    Raises:
        ValueError: If the input direction is not one of the valid direction symbols
    """
    if direction == "^":
        return ">"
    elif direction == ">":
        return "v"
    elif direction == "v":
        return "<"
    elif direction == "<":
        return "^"

In [831]:
def is_valid_position(grid: list, y: int, x: int) -> bool:
    """Verify if a position exists within the grid boundaries."""
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0
    return 0 <= y < rows and 0 <= x < cols

In [832]:
def movement(
    grid: list[list[str]],
    directions: dict,
    start_pos: tuple,
    start_dir: str,
    steps: int = 0,
):
    """
    Simulates movement on a grid based on given directions and rules.

    The function moves a guard on a grid, marking their path and handling obstacles.

    Args:
        grid (List[List[str]]): 2D grid representing the movement area
        directions (Dict[str, Tuple[int, int]]): Dictionary mapping direction names to coordinate changes
        start_pos (Tuple[int, int]): Starting position coordinates (y, x)
        start_dir (str): Initial direction of movement
        steps (int, optional): Number of steps taken so far. Defaults to 0

    Returns:
        Tuple[Union[str, bool], Union[Tuple[int, int], bool], Union[str, bool], Union[int, bool]]:
            Returns a tuple containing:
            - New direction or False if movement is invalid
            - New position or False if movement is invalid
            - Next object encountered or False if movement is invalid
            - Updated step count or False if movement is invalid

    The function handles:
    - Movement within grid boundaries
    - Obstacle detection ('#')
    - Direction changes based on encounters
    - Path marking with 'X'
    """

    def movement2(next_object, next_position):
        if next_object != "#":
            if not is_valid_position(grid, next_position[0], next_position[1]):
                return False, False, False, False
            next_position = (
                next_position + directions[current_direction]
            )  # todo check what is in the next position
            return next_position, current_direction
        if next_object == "#":
            next_direction = change_direction(current_direction)
            next_position = start_pos + directions[next_direction]
            return next_position, next_direction

    print(f"Starting position: {start_pos}, starting direction: {start_dir}")
    current_direction = start_dir
    next_position_y = start_pos[0] + directions[start_dir][0]
    next_position_x = start_pos[1] + directions[start_dir][1]
    next_position = (next_position_y, next_position_x)
    print(f"Next position: {next_position}")
    if is_valid_position(grid, next_position[0], next_position[1]):
        next_object = grid[next_position[0]][
            next_position[1]
        ]  # todo change object symbol to reflect movement
        grid[start_pos[0]][start_pos[1]] = "X"
    else:
        print("Guard has left the area.")
        grid[start_pos[0]][start_pos[1]] = "X"
        return False, False, False, False
    print(f"Next object: {next_object}")
    steps += 1
    next_position, current_direction = movement2(next_object, next_position)
    return current_direction, next_position, next_object, steps


In [833]:
current_direction, next_position, next_object, steps = movement(
    grid, directions, start_pos, start_dir
)

Starting position: (6, 4), starting direction: ^
Next position: (5, 4)
Next object: .


In [834]:
steps = 0
# current_direction, next_position, next_object, steps = movement(grid, directions, start_pos, start_dir)
# todo starting position gets overwritten by the next position
next_position = start_pos
next_object = start_obj

while is_valid_position(grid, next_position[0], next_position[1]) is not False:
    # current_direction, next_position, next_object, steps = movement(grid, directions, start_pos, start_dir)
    while all(
        x is not False for x in [current_direction, next_position, next_object, steps]
    ):
        current_direction, next_position, next_object, steps = movement(
            grid, directions, next_position, current_direction, steps
        )
        print(
            f"Current direction: {current_direction}, next position: {next_position}, next object: {next_object}, steps: {steps}\n"
        )
    else:
        break
    break

Starting position: (6, 4), starting direction: ^
Next position: (5, 4)
Next object: .
Current direction: ^, next position: (5, 4, -1, 0), next object: ., steps: 1

Starting position: (5, 4, -1, 0), starting direction: ^
Next position: (4, 4)
Next object: .
Current direction: ^, next position: (4, 4, -1, 0), next object: ., steps: 2

Starting position: (4, 4, -1, 0), starting direction: ^
Next position: (3, 4)
Next object: .
Current direction: ^, next position: (3, 4, -1, 0), next object: ., steps: 3

Starting position: (3, 4, -1, 0), starting direction: ^
Next position: (2, 4)
Next object: .
Current direction: ^, next position: (2, 4, -1, 0), next object: ., steps: 4

Starting position: (2, 4, -1, 0), starting direction: ^
Next position: (1, 4)
Next object: .
Current direction: ^, next position: (1, 4, -1, 0), next object: ., steps: 5

Starting position: (1, 4, -1, 0), starting direction: ^
Next position: (0, 4)
Next object: #
Current direction: >, next position: (1, 4, -1, 0, 0, 1), n

In [835]:
x_count = 0
for x in range(len(grid)):
    # print(grid[x])
    for xes in grid[x]:
        if xes == "X":
            x_count += 1
x_count

41

#### Puzzle input

In [836]:
def read_input(filename):
    """Reads a grid file and identifies the starting position and direction.

    This function reads a file containing a grid of characters and identifies a starting position
    marked by '^'. The starting position is then converted to an empty space ('.') in the grid.

    Args:
        filename (str): Path to the input file containing the grid.

    Returns:
        tuple: A tuple containing three elements:
            - list[list[str]]: The grid represented as a 2D list of characters
            - tuple[int, int] or None: The starting position as (row, col), or None if not found
            - str or None: The starting direction ('^'), or None if not found

    Example:
        >>> grid, start_pos, start_dir = read_input('input.txt')
        >>> print(start_pos)
        (0, 1)
        >>> print(start_dir)
        '^'
    """
    grid = []
    start_pos = None
    start_dir = None
    start_obj = None  # todo what is in the floor, if its an obstacle etc

    with open(filename) as f:
        for row, line in enumerate(f):
            row_data = list(line.strip())
            # Find starting position if it exists in this row
            if "^" in row_data:
                col = row_data.index("^")
                start_pos = (row, col)
                start_dir = "^"
                row_data[col] = "."  # Replace start position with empty space
                start_obj = row_data[col]
            grid.append(row_data)

    return grid, start_pos, start_dir, start_obj


# Read the input file
grid, start_pos, start_dir, start_obj = read_input("input.txt")

In [837]:
steps = 0
current_direction = start_dir
next_position = start_pos
next_object = start_obj

while is_valid_position(grid, next_position[0], next_position[1]):
    current_direction, next_position, next_object, steps = movement(
        grid, directions, next_position, current_direction, steps
    )
    if False in (current_direction, next_position, next_object, steps):
        break
    print(
        f"Current direction: {current_direction}, next position: {next_position}, next object: {next_object}, steps: {steps}\n"
    )


Starting position: (93, 71), starting direction: ^
Next position: (92, 71)
Next object: .
Current direction: ^, next position: (92, 71, -1, 0), next object: ., steps: 1

Starting position: (92, 71, -1, 0), starting direction: ^
Next position: (91, 71)
Next object: .
Current direction: ^, next position: (91, 71, -1, 0), next object: ., steps: 2

Starting position: (91, 71, -1, 0), starting direction: ^
Next position: (90, 71)
Next object: .
Current direction: ^, next position: (90, 71, -1, 0), next object: ., steps: 3

Starting position: (90, 71, -1, 0), starting direction: ^
Next position: (89, 71)
Next object: .
Current direction: ^, next position: (89, 71, -1, 0), next object: ., steps: 4

Starting position: (89, 71, -1, 0), starting direction: ^
Next position: (88, 71)
Next object: .
Current direction: ^, next position: (88, 71, -1, 0), next object: ., steps: 5

Starting position: (88, 71, -1, 0), starting direction: ^
Next position: (87, 71)
Next object: .
Current direction: ^, nex

In [838]:
x_count = 0
for x in range(len(grid)):
    # print(grid[x])
    for xes in grid[x]:
        if xes == "X":
            x_count += 1
x_count

4433

### Part Two

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and [record](https://adventofcode.com/2018/day/4) the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are _pretty sure_ that adding a single new obstruction _won't_ cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get `<span title="This vulnerability was later fixed by having the guard always turn left instead."><em>`stuck in a loop`</em>`, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know _all_ of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only _`6`_ different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use `O` to mark the new obstruction, `|` to show a position where the guard moves up/down, `-` to show a position where the guard moves left/right, and `+` to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

    ....#.....
    ....+---+#
    ....|...|.
    ..#.|...|.
    ....|..#|.
    ....|...|.
    .#.O^---+.
    ........#.
    #.........
    ......#...

Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

    ....#.....
    ....+---+#
    ....|...|.
    ..#.|...|.
    ..+-+-+#|.
    ..|.|.|.|.
    .#+-^-+-+.
    ......O.#.
    #.........
    ......#...

Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

    ....#.....
    ....+---+#
    ....|...|.
    ..#.|...|.
    ..+-+-+#|.
    ..|.|.|.|.
    .#+-^-+-+.
    .+----+O#.
    #+----+...
    ......#...

Option four, put an alchemical retroencabulator near the bottom left corner:

    ....#.....
    ....+---+#
    ....|...|.
    ..#.|...|.
    ..+-+-+#|.
    ..|.|.|.|.
    .#+-^-+-+.
    ..|...|.#.
    #O+---+...
    ......#...

Option five, put the alchemical retroencabulator a bit to the right instead:

    ....#.....
    ....+---+#
    ....|...|.
    ..#.|...|.
    ..+-+-+#|.
    ..|.|.|.|.
    .#+-^-+-+.
    ....|.|.#.
    #..O+-+...
    ......#...

Option six, put a tank of sovereign glue right next to the tank of universal solvent:

    ....#.....
    ....+---+#
    ....|...|.
    ..#.|...|.
    ..+-+-+#|.
    ..|.|.|.|.
    .#+-^-+-+.
    .+----++#.
    #+----++..
    ......#O..

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are _`6`_ different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. _How many different positions could you choose for this obstruction?_

#### Example

In [839]:
def read_input(filename: str):
    """Reads a grid file and identifies the starting position and direction.

    This function reads a file containing a grid of characters and identifies a starting position
    marked by '^'. The starting position is then converted to an empty space ('.') in the grid.

    Args:
        filename (str): Path to the input file containing the grid.

    Returns:
        tuple: A tuple containing three elements:
            - list[list[str]]: The grid represented as a 2D list of characters
            - tuple[int, int] or None: The starting position as (row, col), or None if not found
            - str or None: The starting direction ('^'), or None if not found
            - str or None: The object in the starting position, or None if not found

    Example:
        >>> grid, start_pos, start_dir, start_obj = read_input('input.txt')
        >>> print(start_pos)
        (0, 1)
        >>> print(start_dir)
        '^'
        >>> print(start_obj)
        '.'
    """
    grid = []
    start_pos = None
    start_dir = None
    start_obj = None

    with open(filename) as f:
        for row, line in enumerate(f):
            row_data = list(line.strip())
            # Find starting position if it exists in this row
            if "^" in row_data:
                col = row_data.index("^")
                start_pos = (row, col)
                start_dir = "^"
                # row_data[col] = "."  # Replace start position with empty space
                start_obj = row_data[col]
            grid.append(row_data)

    return grid, start_pos, start_dir, start_obj


# Read the input file
grid, start_pos, start_dir, start_obj = read_input("example.txt")

In [840]:
def print_grid(grid: list[list]):
    """Prints a 2D grid with borders.

    This function takes a 2D grid (list of lists) and prints it to the console with
    borders around the edges. The grid is formatted with:
    - A top and bottom border made of '+' corners and '-' edges
    - Vertical '|' borders on the left and right sides
    - Single space padding between elements

    Args:
        grid (List[List]): A 2D grid represented as a list of lists where each inner
        list represents a row of elements.

    Example:
        >>> grid = [[1, 2, 3], [4, 5, 6]]
        >>> print_grid(grid)
        +-------+
        | 1 2 3 |
        | 4 5 6 |
        +-------+
    """
    # Get dimensions
    rows = len(grid)
    cols = len(grid[0])

    # Create top border
    border = "+" + "-" * (cols * 2 + 1) + "+"

    print(border)
    # Print each row
    for row in grid:
        print("|", end=" ")
        for cell in row:
            print(cell, end=" ")
        print("|")
    print(border)

In [841]:
def movement(
    grid: list[list],
    directions: dict = {
        "^": (-1, 0),
        ">": (0, 1),
        "v": (1, 0),
        "<": (0, -1),
    },
    start_pos: tuple = (0, 0),
    start_dir: str = "^",
    steps: int = 0,
):
    def update_grid_symbol(
        grid: list[list], position: tuple, direction: str, previous_symbol="."
    ) -> None:
        """
        Updates the symbol at a given position in the grid based on movement direction and previous symbol.

        Args:
            grid (list[list]): 2D grid to be updated
            position (tuple): (y,x) coordinates of position to update
            direction (str): Movement direction symbol (^, v, <, >)
            previous_symbol (str, optional): Symbol at position before update. Defaults to ".".

        Notes:
            - Preserves directional symbols (^, v, <, >)
            - Creates vertical paths (|) for up/down movement
            - Creates horizontal paths (-) for left/right movement
            - Creates crossings (+) when paths intersect
            - No return value as grid is modified in place
        """
        y, x = position[0], position[1]

        # Guard direction symbol
        if previous_symbol in ["^", "v", "<", ">"]:  #! FIXME wrong symbols?
            grid[y][x] = direction
            return

        # Vertical movement
        if direction in ["^", "v"]:
            if previous_symbol in ["-", "+"]:
                grid[y][x] = "+"
            else:
                grid[y][x] = "|"

        # Horizontal movement
        elif direction in ["<", ">"]:
            if previous_symbol in ["|", "+"]:
                grid[y][x] = "+"
            else:
                grid[y][x] = "-"

        # Keep crossings
        if previous_symbol == "+":
            grid[y][x] = "+"

    def movement2(
        next_object: str, next_position: tuple
    ):  # ? should i add current direction as an argument?
        if next_object != "#":
            if not is_valid_position(grid, next_position[0], next_position[1]):
                return False, False, False, False
            # next_position_x = next_position[1] + directions[current_direction][1]
            # next_position_y = next_position[0] + directions[current_direction][0]
            # next_position = (next_position_y, next_position_x)
            next_position = (
                next_position
                + directions[
                    current_direction
                ]  #! FIXME is returning tuple that is not (y, x)
            )  # todo check what is in the next position
            return next_position, current_direction
        if next_object == "#":
            next_direction = change_direction(current_direction)
            next_position = start_pos + directions[next_direction]
            return next_position, next_direction

    def check_right(
        grid: list[list], position: tuple, direction: str
    ) -> (
        bool
    ):  # TODO maybe after every 3 right turns check where an obstacle could be placed
        """Verifies if turning right leads to a loop"""
        next_direction = change_direction(direction)
        next_position_x = position[1] + directions[next_direction][1]
        next_position_y = position[0] + directions[next_direction][0]
        next_position = (next_position_y, next_position_x)

        while is_valid_position(grid, next_position[0], next_position[1]):
            right_turns = 0
            next_object = grid[next_position[0]][next_position[1]]
            if next_object == "#":
                return True  # TODO create check to see if loops when adding an obstacle
            grid[next_position[0]][next_position[1]] = "O"
            next_direction = change_direction(next_direction)
            right_turns += 1
            if right_turns == 3:
                if next_object in ["-", "|", "^"]:
                    grid[next_position[0]][next_position[1]] = "O"
                    right_turns = 0
        return False

    print(f"Starting position: {start_pos}, starting direction: {start_dir}")
    current_direction = start_dir
    current_symbol = grid[start_pos[0]][start_pos[1]]
    next_position_y = start_pos[0] + directions[start_dir][0]
    next_position_x = start_pos[1] + directions[start_dir][1]
    next_position = (next_position_y, next_position_x)
    print(f"Next position: {next_position}")
    if is_valid_position(grid, next_position[0], next_position[1]):
        next_object = grid[next_position[0]][
            next_position[1]
        ]  # todo change object symbol to reflect movement
        grid[start_pos[0]][start_pos[1]] = "X"
        update_grid_symbol(grid, start_pos, current_direction, current_symbol)
    else:
        print("Guard has left the area.")
        grid[start_pos[0]][start_pos[1]] = "X"
        return False, False, False, False, start_pos
    print(f"Next object: {next_object}")
    steps += 1
    next_position, current_direction = movement2(next_object, next_position)
    return current_direction, next_position, next_object, steps, start_pos


In [842]:
# Read the input file
grid, start_pos, start_dir, start_obj = read_input("example.txt")
print_grid(grid)

steps = 0
current_direction = start_dir
next_position = start_pos
next_object = start_obj
obstacle_grid = None

while is_valid_position(grid, next_position[0], next_position[1]):
    current_direction, next_position, next_object, steps, last_pos = movement(
        grid=grid, start_pos=next_position, start_dir=current_direction, steps=steps
    )
    if False in (current_direction, next_position, next_object, steps):
        if obstacle_grid is None:
            obstacle_grid = [row[:] for row in grid]  # Create a deep copy of the grid
            # TODO change symbol of last position before obstacle, add logic to change to '+'
            # if last_pos[2] in [-1, 1] and last_pos[3] == 0:
            #     obstacle_grid[last_pos[0] + last_pos[2]][last_pos[1]] = "|"
            # elif last_pos[3] in [-1, 1] and last_pos[3] == 0:
            #     obstacle_grid[last_pos[0]][last_pos[1] + last_pos[3]] = "-"
            obstacle_grid[last_pos[0]][last_pos[1]] = (
                "O"  # TODO this should be the starting position before guard leaves the area
            )
        print_grid(obstacle_grid)
        break
    print(
        f"Current direction: {current_direction}, next position: {next_position}, next object: {next_object}, steps: {steps}\n"
    )


+---------------------+
| . . . . # . . . . . |
| . . . . . . . . . # |
| . . . . . . . . . . |
| . . # . . . . . . . |
| . . . . . . . # . . |
| . . . . . . . . . . |
| . # . . ^ . . . . . |
| . . . . . . . . # . |
| # . . . . . . . . . |
| . . . . . . # . . . |
+---------------------+
Starting position: (6, 4), starting direction: ^
Next position: (5, 4)
Next object: .
Current direction: ^, next position: (5, 4, -1, 0), next object: ., steps: 1

Starting position: (5, 4, -1, 0), starting direction: ^
Next position: (4, 4)
Next object: .
Current direction: ^, next position: (4, 4, -1, 0), next object: ., steps: 2

Starting position: (4, 4, -1, 0), starting direction: ^
Next position: (3, 4)
Next object: .
Current direction: ^, next position: (3, 4, -1, 0), next object: ., steps: 3

Starting position: (3, 4, -1, 0), starting direction: ^
Next position: (2, 4)
Next object: .
Current direction: ^, next position: (2, 4, -1, 0), next object: ., steps: 4

Starting position: (2, 4, -1, 0), s

#### Claude help

In [843]:
def check_right(grid: list[list], position: tuple, direction: str) -> bool:
    """
    Verifies if placing an obstacle at the current position would create a loop.
    Returns True if a loop would form, False otherwise.
    """
    # Keep track of visited positions and their directions
    visited = set()  # Store (y, x, direction) tuples

    # Start simulation from current position
    current_pos = position
    current_dir = direction

    while True:
        # Create a state tuple that represents current position and direction
        state = (current_pos[0], current_pos[1], current_dir)

        # If we've seen this state before, we've found a loop
        if state in visited:
            return True

        # Add current state to visited set
        visited.add(state)

        # Calculate next position based on current direction
        next_y = current_pos[0] + directions[current_dir][0]
        next_x = current_pos[1] + directions[current_dir][1]
        next_pos = (next_y, next_x)

        # Check if next position is valid
        if not is_valid_position(grid, next_y, next_x):
            return False

        # Check if there's an obstacle ahead
        if grid[next_y][next_x] == "#":
            # Turn right
            current_dir = change_direction(current_dir)
            continue

        # Move forward
        current_pos = next_pos

In [844]:
def find_loop_positions(
    grid: list[list], start_pos: tuple, start_dir: str
) -> list[tuple]:
    """
    Find all positions where placing an obstacle would create a loop.
    Returns list of (y, x) positions.
    """
    loop_positions = []

    # Try each position in the grid
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            # Skip if position already has obstacle or is start position
            if grid[y][x] == "#" or (y, x) == start_pos:
                continue

            # Create a copy of the grid for testing
            test_grid = [row[:] for row in grid]

            # Place test obstacle
            test_grid[y][x] = "#"

            # Check if this creates a loop
            if check_right(test_grid, start_pos, start_dir):
                loop_positions.append((y, x))

    return loop_positions

In [845]:
# Read the input file
grid, start_pos, start_dir, start_obj = read_input("example.txt")

# Find all positions that create loops
loop_positions = find_loop_positions(grid, start_pos, start_dir)

print(f"Found {len(loop_positions)} positions that create loops:")
for pos in loop_positions:
    # Create a grid showing each loop configuration
    loop_grid = [row[:] for row in grid]
    loop_grid[pos[0]][pos[1]] = 'O'
    print(f"\nObstacle at position {pos}:")
    print_grid(loop_grid)

Found 6 positions that create loops:

Obstacle at position (6, 3):
+---------------------+
| . . . . # . . . . . |
| . . . . . . . . . # |
| . . . . . . . . . . |
| . . # . . . . . . . |
| . . . . . . . # . . |
| . . . . . . . . . . |
| . # . O ^ . . . . . |
| . . . . . . . . # . |
| # . . . . . . . . . |
| . . . . . . # . . . |
+---------------------+

Obstacle at position (7, 6):
+---------------------+
| . . . . # . . . . . |
| . . . . . . . . . # |
| . . . . . . . . . . |
| . . # . . . . . . . |
| . . . . . . . # . . |
| . . . . . . . . . . |
| . # . . ^ . . . . . |
| . . . . . . O . # . |
| # . . . . . . . . . |
| . . . . . . # . . . |
+---------------------+

Obstacle at position (7, 7):
+---------------------+
| . . . . # . . . . . |
| . . . . . . . . . # |
| . . . . . . . . . . |
| . . # . . . . . . . |
| . . . . . . . # . . |
| . . . . . . . . . . |
| . # . . ^ . . . . . |
| . . . . . . . O # . |
| # . . . . . . . . . |
| . . . . . . # . . . |
+---------------------+

Obstacle

In [856]:
def simulate_guard_path(
    grid: list[list], start_pos: tuple, start_dir: str, simulate: bool = True
) -> tuple[bool, list[list] | None]:
    """
    Simulates guard movement and optionally updates grid with path symbols.
    Returns:
        tuple: (has_loop: bool, path_grid: list[list] | None)
        - has_loop: True if a loop was found
        - path_grid: Updated grid with path markings if simulate=True and loop found, None otherwise
    """
    # Create a working copy of the grid only if simulating
    path_grid = [row[:] for row in grid] if simulate else None

    # Keep track of visited positions and their directions
    visited = set()  # Store (y, x, direction) tuples
    current_pos = start_pos
    current_dir = start_dir

    def update_grid_symbol(
        pos: tuple, direction: str, previous_symbol: str = "."
    ) -> None:
        """Updates grid symbols based on movement direction"""
        if not simulate:
            return

        y, x = pos
        # Guard direction symbol - preserve only the current position
        if previous_symbol in ["^", "v", "<", ">"]:
            path_grid[y][x] = direction
            return

        # Vertical movement
        if direction in ["^", "v"]:
            if previous_symbol in ["-", "+"]:
                path_grid[y][x] = "+"
            else:
                path_grid[y][x] = "|"

        # Horizontal movement
        elif direction in ["<", ">"]:
            if previous_symbol in ["|", "+"]:
                path_grid[y][x] = "+"
            else:
                path_grid[y][x] = "-"

        # Keep crossings
        if previous_symbol == "+":
            path_grid[y][x] = "+"

    while True:
        # Create a state tuple that represents current position and direction
        state = (current_pos[0], current_pos[1], current_dir)

        # If we've seen this state before, we've found a loop
        if state in visited:
            return True, path_grid

        # Add current state to visited set
        visited.add(state)

        # Update grid symbol at current position if simulating
        if simulate:
            prev_symbol = path_grid[current_pos[0]][current_pos[1]]
            update_grid_symbol(current_pos, current_dir, prev_symbol)

        # Calculate next position based on current direction
        next_y = current_pos[0] + directions[current_dir][0]
        next_x = current_pos[1] + directions[current_dir][1]

        # Check if next position is valid
        if not is_valid_position(grid, next_y, next_x):
            return False, None

        # Check if there's an obstacle ahead
        if grid[next_y][next_x] == "#":
            # Turn right
            current_dir = change_direction(current_dir)
            continue

        # Move forward
        current_pos = (next_y, next_x)


def find_loop_positions(
    grid: list[list], start_pos: tuple, start_dir: str, simulate: bool = True
) -> list[tuple] | list[tuple[tuple, list[list]]]:
    """
    Find all positions where placing an obstacle would create a loop.
    Args:
        grid: The input grid
        start_pos: Starting position
        start_dir: Starting direction
        simulate: If True, return positions and path grids; if False, return only positions
    Returns:
        If simulate=True: list of tuples (position, path_grid)
        If simulate=False: list of positions (y, x)
    """
    results = []

    # Try each position in the grid
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            # Skip if position already has obstacle or is start position
            if grid[y][x] == "#" or (y, x) == start_pos:
                continue

            # Create a copy of the grid for testing
            test_grid = [row[:] for row in grid]

            # Place test obstacle
            test_grid[y][x] = "#"

            # Check for loop and optionally get path grid
            has_loop, path_grid = simulate_guard_path(
                test_grid, start_pos, start_dir, simulate
            )

            # If we found a loop
            if has_loop:
                if simulate:
                    # Mark the obstacle with 'O' in the final visualization
                    path_grid[y][x] = "O"
                    results.append(((y, x), path_grid))
                else:
                    # Just store the coordinates when not simulating
                    results.append((y, x))  # No grid, just the position tuple

    return results


def main(filename: str = "example.txt", simulate: bool = False):
    # Read the input file
    grid, start_pos, start_dir, start_obj = read_input(filename)

    if not simulate:
        # Just get the positions without visualization
        loop_positions = find_loop_positions(grid, start_pos, start_dir, simulate=False)
        print(f"Found {len(loop_positions)} positions that create loops.")
        print("Positions:", loop_positions)
    else:
        # Get full visualizations
        loop_configs = find_loop_positions(grid, start_pos, start_dir, simulate=True)
        print(f"Found {len(loop_configs)} positions that create loops.")

        # Print each loop configuration
        for i, (pos, path_grid) in enumerate(loop_configs, 1):
            print(f"\nOption {i}, obstacle at position {pos}:")
            print_grid(path_grid)


if __name__ == "__main__":
    main(simulate=True)

Found 6 positions that create loops.

Option 1, obstacle at position (6, 3):
+---------------------+
| . . . . # . . . . . |
| . . . . + - - - + # |
| . . . . | . . . | . |
| . . # . | . . . | . |
| . . . . | . . # | . |
| . . . . | . . . | . |
| . # . O < - - - + . |
| . . . . . . . . # . |
| # . . . . . . . . . |
| . . . . . . # . . . |
+---------------------+

Option 2, obstacle at position (7, 6):
+---------------------+
| . . . . # . . . . . |
| . . . . + - - - + # |
| . . . . | . . . | . |
| . . # . | . . . | . |
| . . + - + - + # | . |
| . . | . | . | . | . |
| . # + - < - + - + . |
| . . . . . . O . # . |
| # . . . . . . . . . |
| . . . . . . # . . . |
+---------------------+

Option 3, obstacle at position (7, 7):
+---------------------+
| . . . . # . . . . . |
| . . . . + - - - + # |
| . . . . | . . . | . |
| . . # . | . . . | . |
| . . + - + - + # | . |
| . . | . | . | . | . |
| . # + - < - + - + . |
| . + - - - - + O # . |
| # + - - - - + . . . |
| . . . . . . # . . . |
+--

#### Executing the puzzle

In [854]:
main('input.txt')

Found 1516 positions that create loops.
Positions: [(1, 22), (1, 26), (2, 22), (2, 42), (2, 44), (2, 45), (2, 51), (2, 52), (2, 54), (2, 55), (2, 56), (2, 57), (2, 60), (2, 62), (2, 63), (2, 65), (2, 67), (2, 68), (2, 69), (2, 71), (2, 72), (2, 74), (2, 75), (2, 76), (2, 82), (2, 84), (2, 85), (3, 22), (3, 39), (3, 116), (3, 117), (3, 118), (3, 124), (3, 127), (4, 22), (5, 115), (6, 27), (6, 28), (6, 36), (6, 38), (6, 41), (6, 47), (6, 53), (6, 54), (6, 56), (6, 57), (6, 64), (6, 66), (6, 72), (6, 77), (6, 83), (7, 27), (7, 39), (8, 9), (8, 27), (8, 39), (8, 46), (8, 51), (8, 53), (8, 54), (8, 55), (8, 56), (8, 58), (8, 59), (8, 60), (8, 62), (8, 63), (8, 65), (8, 67), (8, 68), (8, 69), (8, 71), (8, 76), (8, 79), (8, 82), (8, 84), (9, 6), (9, 25), (9, 29), (9, 32), (9, 33), (9, 34), (9, 35), (9, 42), (9, 44), (9, 45), (9, 51), (9, 53), (9, 54), (9, 55), (9, 58), (9, 62), (9, 63), (9, 65), (9, 67), (9, 68), (9, 69), (9, 71), (9, 76), (9, 79), (9, 82), (9, 84), (9, 86), (9, 91), (10, 27)